In [1]:
#필요한 라이브러리 설치
!pip install langchain
!pip install langchain_openai
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 280.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [2]:
import os
import openai

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

openai.api_key = os.getenv("sk-proj-JmXIBJ4HZfgQ47BuKP0uT3BlbkFJnY7JKAkk3p6Z6LHHUmtL")

SecretNotFoundError: Secret OPENAI_API_KEY does not exist.

## 체인

-  | (체인)
- 여러개의 작업을 연결하여 수행
- 여러번 정의 X -> 한번에 엮어서 순차적으로 수행 가능

In [ ]:
#message.choice[0].content X -> 내용만 가져올 수 있게 도와주는 함수
from langchain_core.output_parsers import StrOutputParser

#대화형 프롬프트를 정의
from langchain_core.prompts import ChatPromptTemplate

#openai와 대화할 수 있게 하는 클래스
from langchain_openai import ChatOpenAI

In [ ]:
#채팅 모델 정의(openai에 질의응답을 전달할 수 있도록 함)
chat_model = ChatOpenAI()

#챗프롬프트템플릿
#동일한 내용에 대해 전달할 때 {topic}만 바꿔주면 된다.
chat_prompt_template = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

#콘텐츠만 추출하도록 도와주는
output_parser = StrOutputParser()

#체인 정의
chain = chat_prompt_template | chat_model | output_parser

chain.invoke({"topic":"programmer"})

In [ ]:
#챗프롬프트템플릿
#동일한 내용에 대해 전달할 때 {topic}만 바꿔주면 된다.
chat_prompt_template = ChatPromptTemplate.from_template("Please explain the {city}'s history")

chain = chat_prompt_template | ChatOpenAI() | StrOutputParser()
chain.invoke({"city" : "seoul"})

## Runnable 객체

- RunnablePassthrough : 입력값을 - 출력값으로 전달
- RunnableParallel : 여러개의 Runnable을 병렬로 실행
- RunnableLambda : 람다 함수나, 임의의 함수를 사용할 수 있게 감싸줌

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_template("What's your favorite {thing}?")

chat_model = ChatOpenAI()
output_parser = StrOutputParser()

#체인의 순서가 바뀌어도 되는가?
chain = {"thing" : RunnablePassthrough()} | chat_prompt_template | chat_model | output_parser

In [ ]:
chain.invoke("movie")

In [ ]:
chain.invoke("color")

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_template("What's your favorite {thing}?")

chat_model = ChatOpenAI()
output_parser = StrOutputParser()

#체인의 순서가 바뀌어도 되는가?
chain = RunnablePassthrough() | chat_prompt_template | chat_model | output_parser
chain.invoke({"thing":"city"})

## 러너블 객체 활용하기

1. 객체 방식으로 접근하기

##### dunder메서드(매직메서드)     
chain = a.__or__(b)    
__init__

2. 파이프라인 문법으로 접근하기          
chain = a | b

In [ ]:
#Runnable이라는 '객체' Runnable()
#내가 만든 함수를 runnable 체인 사이에 끼우고 싶어!
class Runnable:

  #class가 처음 정의될 때
  #runnable을 만들때, 함수를 하나 전달해서 함수를 'runnable'객체로 만드는 과정
  def __init__(self, func):
    self.func = func

  #체인 문법을 사용하기 위해
  # 함수A | 함수B
  def __or__(self, other):

    # *args = 어떤 파라미터를 전해주던지, 받아들일 수 있게
    def chained_func(*args, **kwargs):
      return other(self.func(*args, **kwargs))

    return Runnable(chained_func)

  #호출 가능하도록
  def __call__(self, *args, **kwargs):
    return self.func(*args, **kwargs)

In [ ]:
# *args, **kwargs 설명

def add(*args):
  for a in args:
    print(a)

In [ ]:
add(1, 2, 4, 5, 7, 8, 3, 6, 8, 'string')

In [ ]:
def pr(**kwargs):
  for k, v in kwargs.items():
    print(f"{k} = {v}")

In [ ]:
pr(a=1)

In [ ]:
def add_five(x):
  return x+5

def multiply_by_two(x):
  return x*2

In [ ]:
add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

In [ ]:
add_five

In [ ]:
chain = add_five | multiply_by_two
chain(3)

In [ ]:
chain = add_five.__or__(multiply_by_two)
chain(3)

### RunnablePassthrough

In [ ]:
from langchain_core.runnables import RunnablePassthrough

templates = "Please generate three sentences in a dialougue in {lang} on the topic {topic}"

chat_prompt_template = ChatPromptTemplate.from_template(templates)

chain = chat_prompt_template | chat_model | StrOutputParser()

output = chain.invoke({"lang" : "french", "topic": "vacance"})
print(output)

In [ ]:
# 괄호 안의 _는 전달되는 매개변수를 무시하라는 뜻
def return_lang(_):
  return "french"

def return_topic(_):
  return "olympic"

In [ ]:
chain = RunnablePassthrough.assign(lang = return_lang) | chat_prompt_template | chat_model | StrOutputParser()

output = chain.invoke({"topic" : "summer"})
print(output)